In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE


In [2]:
meio_posicao= pd.read_excel('meio_posicao_queen.xlsx', sheet_name=0)
df = pd.DataFrame(meio_posicao)

In [3]:
# ==== 2. Separar features e target ====
X = df[["bom","ruim","media","idade","tempfranquia"]]
y = df["vencedora"]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


In [5]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)


In [6]:
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=6,
    random_state=42
)
rf.fit(X_res, y_res)

RandomForestClassifier(max_depth=6, n_estimators=200, random_state=42)

In [7]:
y_pred = rf.predict(X_test)
print("Matriz de confusão:\n", confusion_matrix(y_test, y_pred))
print("\nRelatório de classificação:\n", classification_report(y_test, y_pred))


Matriz de confusão:
 [[111  24]
 [  4  13]]

Relatório de classificação:
               precision    recall  f1-score   support

           0       0.97      0.82      0.89       135
           1       0.35      0.76      0.48        17

    accuracy                           0.82       152
   macro avg       0.66      0.79      0.68       152
weighted avg       0.90      0.82      0.84       152



In [8]:
importances = pd.DataFrame({
    "feature": X.columns,
    "importancia": rf.feature_importances_
}).sort_values(by="importancia", ascending=False)

print("\nImportância das variáveis:\n", importances)


Importância das variáveis:
         feature  importancia
2         media     0.393473
1          ruim     0.264477
0           bom     0.129856
4  tempfranquia     0.121913
3         idade     0.090281


In [9]:
# Probabilidades de cada participante no teste
probs = rf.predict_proba(X_test)

# Colocar em um DataFrame junto com o nome da queen
resultados = pd.DataFrame({
    "queen": df.loc[X_test.index, "queen"],
    "prob_nao_vencedora": probs[:,0],
    "prob_vencedora": probs[:,1],
    "real": y_test.values
}).sort_values(by="prob_vencedora", ascending=False)

print(resultados)


                      queen  prob_nao_vencedora  prob_vencedora  real
354       Melissa Bianchini            0.078941        0.921059     0
356  Silvana Della Magliana            0.094458        0.905542     0
497              Kween Kong            0.098715        0.901285     0
46             BenDeLaCreme            0.116558        0.883442     0
330          La Bella Vampi            0.188120        0.811880     0
..                      ...                 ...             ...   ...
462          Spicy Sunshine            0.990970        0.009030     0
292            Ivanna Drink            0.992307        0.007693     0
259          Madame Madness            0.992664        0.007336     0
146                  Megami            0.992668        0.007332     0
467                Shortgun            0.995034        0.004966     0

[152 rows x 4 columns]
